<a href="https://colab.research.google.com/github/fboldt/aulas-am-bsi/blob/main/aula12a_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip titanic.zip

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [2]:
import pandas as pd

df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.dtypes

,0
PassengerId,int64
Survived,int64
Pclass,int64
Name,object
Sex,object
Age,float64
SibSp,int64
Parch,int64
Ticket,object
Fare,float64


In [12]:
y = df['Survived']
X = df.drop('Survived', axis=1)

In [13]:
len(y)

891

In [14]:
for col in X.columns:
    print(f"{col:>12} {len(X[col].unique()):4}  {X[col].dtype}")

 PassengerId  891  int64
      Pclass    3  int64
        Name  891  object
         Sex    2  object
         Age   89  float64
       SibSp    7  int64
       Parch    7  int64
      Ticket  681  object
        Fare  248  float64
       Cabin  148  object
    Embarked    4  object


In [15]:
caracteristicas_indesejadas = ['PassengerId', 'Name', 'Ticket', 'Cabin']
Xdrop = X.drop(caracteristicas_indesejadas, axis=1)
Xdrop.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [16]:
Xnum = Xdrop.select_dtypes('number')
Xnum.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [17]:
for col in Xnum.columns:
    print(f"{col:>12} {X[col].isnull().sum():4}")

      Pclass    0
         Age  177
       SibSp    0
       Parch    0
        Fare    0


In [18]:
Xnum[-5:]

,Pclass,Age,SibSp,Parch,Fare
886,2,27.0,0,0,13.00
887,1,19.0,0,0,30.00
888,3,NaN,1,2,23.45
889,1,26.0,0,0,30.00
890,3,32.0,0,0,7.75


In [20]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
XnumTratado = imputer.fit_transform(Xnum)
XnumTratado[-5:]

array([[ 2.  , 27.  ,  0.  ,  0.  , 13.  ],
       [ 1.  , 19.  ,  0.  ,  0.  , 30.  ],
       [ 3.  , 28.  ,  1.  ,  2.  , 23.45],
       [ 1.  , 26.  ,  0.  ,  0.  , 30.  ],
       [ 3.  , 32.  ,  0.  ,  0.  ,  7.75]])

In [21]:
Xcat = Xdrop.select_dtypes('object')
Xcat.columns

Index(['Sex', 'Embarked'], dtype='object')

In [22]:
for col in Xcat.columns:
    print(f"{col:>12} {X[col].isnull().sum():4}")

         Sex    0
    Embarked    2


In [23]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')
XcatTratado = imputer.fit_transform(Xcat)
XcatTratado[-5:]

array([['male', 'S'],
       ['female', 'S'],
       ['female', 'S'],
       ['male', 'C'],
       ['male', 'Q']], dtype=object)

In [25]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
XcatTratadoHot = encoder.fit_transform(XcatTratado)
XcatTratadoHot

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1782 stored elements and shape (891, 5)>

In [26]:
XcatTratadoHot.toarray()[-5:]

array([[0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0.]])

In [27]:
import numpy as np

Xtratado = np.c_[XnumTratado, XcatTratadoHot.toarray()]
Xtratado[-5:]

array([[ 2.  , 27.  ,  0.  ,  0.  , 13.  ,  0.  ,  1.  ,  0.  ,  0.  ,
         1.  ],
       [ 1.  , 19.  ,  0.  ,  0.  , 30.  ,  1.  ,  0.  ,  0.  ,  0.  ,
         1.  ],
       [ 3.  , 28.  ,  1.  ,  2.  , 23.45,  1.  ,  0.  ,  0.  ,  0.  ,
         1.  ],
       [ 1.  , 26.  ,  0.  ,  0.  , 30.  ,  0.  ,  1.  ,  1.  ,  0.  ,
         0.  ],
       [ 3.  , 32.  ,  0.  ,  0.  ,  7.75,  0.  ,  1.  ,  0.  ,  1.  ,
         0.  ]])

In [28]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosDesejados(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasIndesejadas = ['PassengerId', 'Name', 'Ticket', 'Cabin']
        return self
    def transform(self, X, y=None):
        return X.drop(self.colunasIndesejadas, axis=1)

Xdrop = AtributosDesejados().fit_transform(X)
Xdrop.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [29]:
class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X.select_dtypes(include='number')

Xnum = AtributosNumericos().fit_transform(Xdrop)
Xnum.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

pipenum = Pipeline([
    ('atributos_numericos', AtributosNumericos()),
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

XnumTratado = pipenum.fit_transform(Xdrop)
XnumTratado

array([[ 0.82737724, -0.56573646,  0.43279337, -0.47367361, -0.50244517],
       [-1.56610693,  0.66386103,  0.43279337, -0.47367361,  0.78684529],
       [ 0.82737724, -0.25833709, -0.4745452 , -0.47367361, -0.48885426],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337,  2.00893337, -0.17626324],
       [-1.56610693, -0.25833709, -0.4745452 , -0.47367361, -0.04438104],
       [ 0.82737724,  0.20276197, -0.4745452 , -0.47367361, -0.49237783]])

In [31]:
class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X.select_dtypes(include='object')

Xcat = AtributosCategoricos().fit_transform(Xdrop)
Xcat.columns

Index(['Sex', 'Embarked'], dtype='object')

In [32]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

pipecat = Pipeline([
    ('atributos_categoricos', AtributosCategoricos()),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())
])

XcatTratado = pipecat.fit_transform(Xdrop)
XcatTratado

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1782 stored elements and shape (891, 5)>

In [40]:
from sklearn.pipeline import FeatureUnion

unecaracteristicas = FeatureUnion([
    ('pipenum', pipenum),
    ('pipecat', pipecat)
])

Xtratado = unecaracteristicas.fit_transform(Xdrop)
Xtratado

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6237 stored elements and shape (891, 10)>

In [41]:
Xtratado.toarray()

array([[ 0.82737724, -0.56573646,  0.43279337, ...,  0.        ,
         0.        ,  1.        ],
       [-1.56610693,  0.66386103,  0.43279337, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82737724, -0.25833709, -0.4745452 , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337, ...,  0.        ,
         0.        ,  1.        ],
       [-1.56610693, -0.25833709, -0.4745452 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82737724,  0.20276197, -0.4745452 , ...,  0.        ,
         1.        ,  0.        ]])

In [42]:
preproc = Pipeline([
    ('atributos_desejados', AtributosDesejados()),
    ('unecaracteristicas', unecaracteristicas)
])

Xtratado = preproc.fit_transform(X)
Xtratado

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6237 stored elements and shape (891, 10)>

In [46]:
from sklearn.ensemble import RandomForestClassifier

rf = Pipeline([
    ('preproc', preproc),
    ('classificador', RandomForestClassifier())
])

rf.fit(X, y)

Pipeline(steps=[('preproc',
                 Pipeline(steps=[('atributos_desejados', AtributosDesejados()),
                                 ('unecaracteristicas',
                                  FeatureUnion(transformer_list=[('pipenum',
                                                                  Pipeline(steps=[('atributos_numericos',
                                                                                   AtributosNumericos()),
                                                                                  ('imputer',
                                                                                   SimpleImputer(strategy='median')),
                                                                                  ('scaler',
                                                                                   StandardScaler())])),
                                                                 ('pipecat',
                                                                  Pipeline(steps=[('atributos_categoricos',
                                                                                   AtributosCategoricos()),
                                                                                  ('imputer',
                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                  ('encoder',
                                                                                   OneHotEncoder())]))]))])),
                ('classificador', RandomForestClassifier())])

In [48]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from pprint import pprint

scores = cross_validate(rf, X, y,
                        cv=StratifiedKFold(shuffle=True, random_state=42))
pprint(scores)
print(scores['test_score'].mean())

{'fit_time': array([0.33948016, 0.35005164, 0.34361744, 0.34287596, 0.3653667 ]),
 'score_time': array([0.02153087, 0.02118802, 0.02249718, 0.02065229, 0.02107739]),
 'test_score': array([0.84357542, 0.79213483, 0.79775281, 0.82022472, 0.84269663])}
0.8192768815516918


In [49]:
from sklearn.neighbors import KNeighborsClassifier

knn = Pipeline([
    ('preproc', preproc),
    ('classificador', KNeighborsClassifier())
])

scores = cross_validate(knn, X, y,
                        cv=StratifiedKFold(shuffle=True, random_state=42))
pprint(scores)
print(scores['test_score'].mean())

{'fit_time': array([0.01854515, 0.01644588, 0.0173533 , 0.01553798, 0.01501751]),
 'score_time': array([0.0241797 , 0.01775002, 0.01776218, 0.01607132, 0.016186  ]),
 'test_score': array([0.81564246, 0.80337079, 0.78651685, 0.83707865, 0.8258427 ])}
0.8136902893729208


In [51]:
from sklearn.neural_network import MLPClassifier

mlp = Pipeline([
    ('preproc', preproc),
    ('classificador', MLPClassifier(max_iter=2000))
])

scores = cross_validate(mlp, X, y,
                        cv=StratifiedKFold(shuffle=True, random_state=42))
pprint(scores)
print(scores['test_score'].mean())

{'fit_time': array([4.84134245, 4.54279542, 4.08070421, 5.54814839, 4.21732521]),
 'score_time': array([0.01376009, 0.01152945, 0.01022196, 0.00986457, 0.00840354]),
 'test_score': array([0.81564246, 0.83146067, 0.78089888, 0.85393258, 0.80898876])}
0.8181846713953925


In [52]:
!pip install optuna -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 13.1 MB/s eta 0:00:00


In [53]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

def objective(trial):
    # Define hyperparameters to tune
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_split = trial.suggest_float('min_samples_split', 0.1, 1.0)
    min_samples_leaf = trial.suggest_float('min_samples_leaf', 0.1, 0.5)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    # Create the Random Forest classifier with suggested hyperparameters
    rf = Pipeline([
        ('preproc', preproc),
        ('classificador', RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            criterion=criterion,
            random_state=42
        ))
    ])

    # Evaluate the model using cross-validation
    score = cross_val_score(rf, X, y, cv=5, scoring='accuracy').mean()
    return score

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best parameters and best score
print("Best parameters found: ", study.best_params)
print("Best cross-validation accuracy: ", study.best_value)

[I 2025-10-02 15:38:59,145] A new study created in memory with name: no-name-684b21bf-9ec4-4f33-aed0-f1bbfff50390
[I 2025-10-02 15:39:01,733] Trial 0 finished with value: 0.6161634548992531 and parameters: {'n_estimators': 242, 'max_depth': 5, 'min_samples_split': 0.7104032836334176, 'min_samples_leaf': 0.43551097110012604, 'criterion': 'entropy'}. Best is trial 0 with value: 0.6161634548992531.
[I 2025-10-02 15:39:03,109] Trial 1 finished with value: 0.6161634548992531 and parameters: {'n_estimators': 144, 'max_depth': 29, 'min_samples_split': 0.7882142015293178, 'min_samples_leaf': 0.24057354294945552, 'criterion': 'entropy'}. Best is trial 0 with value: 0.6161634548992531.
[I 2025-10-02 15:39:04,499] Trial 2 finished with value: 0.6161634548992531 and parameters: {'n_estimators': 144, 'max_depth': 15, 'min_samples_split': 0.47834263213755535, 'min_samples_leaf': 0.30476172115072575, 'criterion': 'entropy'}. Best is trial 0 with value: 0.6161634548992531.
[I 2025-10-02 15:39:05,752] 

Best parameters found:  {'n_estimators': 156, 'max_depth': 27, 'min_samples_split': 0.15986276672728805, 'min_samples_leaf': 0.1509772684307556, 'criterion': 'gini'}
Best cross-validation accuracy:  0.787853869813571


In [67]:
from sklearn.linear_model import LogisticRegression

lr = Pipeline([
    ('preproc', preproc),
    ('classificador', LogisticRegression())
])

scores = cross_validate(fslr, X, y,
                        cv=StratifiedKFold(shuffle=True, random_state=42))
pprint(scores)
print(scores['test_score'].mean())

{'fit_time': array([0.0920465 , 0.04477143, 0.02722788, 0.0300386 , 0.03461027]),
 'score_time': array([0.01598144, 0.0134244 , 0.01866317, 0.01229453, 0.01184678]),
 'test_score': array([0.77094972, 0.80337079, 0.79213483, 0.78089888, 0.82022472])}
0.7935157868307073


In [71]:
import optuna
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

def objective_lr(trial):
    # Define hyperparameters to tune for Logistic Regression
    C = trial.suggest_loguniform('C', 1e-4, 1e4)
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    # Create the Logistic Regression classifier with suggested hyperparameters
    lr_tuned = Pipeline([
        ('preproc', preproc),
        ('classificador', LogisticRegression(
            C=C,
            solver=solver,
            penalty=penalty,
            random_state=42,
            max_iter=1000 # Increased max_iter for convergence
        ))
    ])

    # Evaluate the model using cross-validation
    # Use StratifiedKFold for cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(lr_tuned, X, y, cv=cv, scoring='accuracy').mean()
    return score

# Create a study object and optimize the objective function
study_lr = optuna.create_study(direction='maximize')
study_lr.optimize(objective_lr, n_trials=100) # You can adjust the number of trials

# Print the best parameters and best score
print("Best parameters found for Logistic Regression: ", study_lr.best_params)
print("Best cross-validation accuracy with tuned Logistic Regression: ", study_lr.best_value)

[I 2025-10-02 15:55:40,326] A new study created in memory with name: no-name-5744dc95-1fa8-46f0-93fc-b7a61357600e
/tmp/ipython-input-4074769522.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e4)
[I 2025-10-02 15:55:40,489] Trial 0 finished with value: 0.7923796371853618 and parameters: {'C': 0.15607346190270163, 'solver': 'saga', 'penalty': 'l1'}. Best is trial 0 with value: 0.7923796371853618.
/tmp/ipython-input-4074769522.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e4)
[I 2025-10-02 15:55:40,895] Trial 1 finished with value: 0.6161634548992531 and parameters: {'C': 0.0001111

Best parameters found for Logistic Regression:  {'C': 0.0348147742020565, 'solver': 'saga', 'penalty': 'l2'}
Best cross-validation accuracy with tuned Logistic Regression:  0.8047203565375682


In [72]:
test = pd.read_csv('test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [73]:
ypred = rf.predict(test)
ypred

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [74]:
submission = pd.read_csv('gender_submission.csv')
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [76]:
submission['Survived'] = ypred
submission.to_csv('rf-submission.csv', index=False)